In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATADIR_chr='/users/mtaranov/3D_fromATAC_by_chr/labels/'
DATADIR='/users/mtaranov/3D_fromATAC/data/labels/'
data_file = DATADIR_chr+'all_atac_1kb_contacts_w_labels.txt'

In [ ]:
data=np.loadtxt(data_file, dtype=str)

In [ ]:
X=data[:,:6]
y=data[:,6]
y=y.reshape((y.shape[0],1))
indx=data[:,7:9]

In [ ]:
def binarize(matrix):
    copy_matrix=copy.copy(matrix)
    copy_matrix[copy_matrix > 0] = 1
    return copy_matrix

In [ ]:
import copy
def get_pairs_distance_matched(X, y, indx, min_dist, max_dist, dist_step, imbalance_ratio):

    def subsample_indx(indecies, size, imbalance_ratio):
        indecies_shuffled=copy.copy(indecies)
        np.random.shuffle(indecies_shuffled)
        num_subsampled = size*imbalance_ratio
        if num_subsampled > len(indecies[0]):
            print '    Error: Not enough to subsample'
            exit
        #print indecies_shuffled[0].shape
        #print indecies_shuffled[0][:num_subsampled].shape
        else:
            return indecies_shuffled[0][:num_subsampled]

    neg_indxs = np.where(y.astype(float)==0)[0]
    pos_indxs = np.where(y.astype(float)>=10)[0]
    X_pos=X[pos_indxs]
    X_neg=X[neg_indxs]
    y_pos=y[pos_indxs]
    y_neg=y[neg_indxs]
    indx_pos=indx[pos_indxs]
    indx_neg=indx[neg_indxs]

    print "total pos=", X_pos.shape
    print "total neg=", X_neg.shape
    thres1=min_dist+dist_step
    thres2=min_dist

    X_new=np.empty(([0,X.shape[1]]))
    y_new=np.empty(([0,y.shape[1]]))
    indx_new=np.empty(([0,indx.shape[1]]))

    while thres1 <= max_dist:
        print 'distance window: ', '[', thres2, ',', thres1, ']'
        neg_indx_at_dist=np.where((abs(X_neg[:,1].astype(int) - X_neg[:,4].astype(int)) <= thres1) & (abs(X_neg[:,1].astype(int) - X_neg[:,4].astype(int)) >= thres2))
        pos_indx_at_dist=np.where((abs(X_pos[:,1].astype(int) - X_pos[:,4].astype(int)) <= thres1) & (abs(X_pos[:,1].astype(int) - X_pos[:,4].astype(int)) >= thres2))
        if len(pos_indx_at_dist[0])> len(neg_indx_at_dist[0]):
            #print 'more pos than neg'
            print 'number of pos at distance=:',  len(pos_indx_at_dist[0])
            print 'number of neg at distance=:',  len(neg_indx_at_dist[0])
            indx_subsampled=subsample_indx(pos_indx_at_dist, len(neg_indx_at_dist[0]), imbalance_ratio)
            new_pos_indx_at_dist=indx_subsampled
            new_neg_indx_at_dist=neg_indx_at_dist[0]

        else:
            #print 'more neg than pos'
            print 'number of pos at distance=',  len(pos_indx_at_dist[0])
            print 'number of neg at distance=',  len(neg_indx_at_dist[0])
            indx_subsampled=subsample_indx(neg_indx_at_dist, len(pos_indx_at_dist[0]), imbalance_ratio)
            new_pos_indx_at_dist=pos_indx_at_dist[0]
            new_neg_indx_at_dist=indx_subsampled

        y_pos_at_dist=y_pos[new_pos_indx_at_dist]
        y_neg_at_dist=y_neg[new_neg_indx_at_dist]
        X_pos_at_dist=X_pos[new_pos_indx_at_dist]
        X_neg_at_dist=X_neg[new_neg_indx_at_dist]
        indx_pos_at_dist=indx_pos[new_pos_indx_at_dist]
        indx_neg_at_dist=indx_neg[new_neg_indx_at_dist]

        y_at_dist=np.concatenate((y_pos_at_dist, y_neg_at_dist))
        X_at_dist=np.concatenate((X_pos_at_dist, X_neg_at_dist))
        indx_at_dist=np.concatenate((indx_pos_at_dist, indx_neg_at_dist))

        print 'labels at dist: ', y_at_dist.shape
        print 'data at dist: ', X_at_dist.shape
        print 'indx at dist: ', indx_at_dist.shape

        indx_new=np.concatenate((indx_new, indx_at_dist))
        X_new=np.concatenate((X_new, X_at_dist))
        y_new=np.concatenate((y_new, y_at_dist))


        #print X_new.shape, X_at_dist.shape
        #print y_new.shape, y_at_dist.shape
        #print indx_new.shape, indx_at_dist.shape

        #print "# of neg:", np.where(y_at_dist==0)[0].shape
        #print "# of pos:", np.where(y_at_dist==1)[0].shape
        #thres2=thres1+min_dist
        thres2=thres1
        thres1=thres1+dist_step

    return X_new, y_new, indx_new

In [ ]:
X_new, y_new, indx_new = get_pairs_distance_matched(X, y, indx, 10000, 2000000, 10000, 1)

In [ ]:
X_new.shape, y_new.shape, indx_new.shape

In [ ]:
plt.hist(abs(X_new[np.where(y_new.astype(float)==0)[0]][:,1].astype(int)-X_new[np.where(y_new.astype(float)==0)[0]][:,4].astype(int)))

In [ ]:
plt.hist(abs(X_new[np.where(y_new.astype(float)!=0)[0]][:,1].astype(int)-X_new[np.where(y_new.astype(float)!=0)[0]][:,4].astype(int)))

In [ ]:
np.savetxt(DATADIR+'site1.bed', X_new[:,:3], delimiter="\t", fmt="%s") 
np.savetxt(DATADIR+'site2.bed', X_new[:,3:], delimiter="\t", fmt="%s") 
#np.savetxt(DATADIR+'labels.txt', y_new, delimiter="\t", fmt="%s") 
np.savetxt(DATADIR+'indx.bed', indx_new, delimiter="\t", fmt="%s") 

In [ ]:
np.save(DATADIR+'labels.npy', binarize(y_new.astype(float)).astype(float))